In [50]:
import cv2
import pandas as pd
import numpy as np

def oct_flash(original_array:np.array, input_loc = []) -> tuple:

    if input_loc:
        coord = input_loc
    else:
        # Find 10's
        coord = np.where(original_array == 10)

    flash_count = len(coord[0])
    # set 10's to 0
    original_array = np.where(original_array == 10, 0, original_array)
    cascade_flashed = (np.array([],dtype='int32'),np.array([],dtype='int32'))

    for y,x in zip(coord[0], coord[1]):

        # Find roi and increment neighbours by 1 if not equal to 0
        roi = original_array[y-1:y+2, x-1:x+2]
        roi_plus1 = np.where(roi != 0, roi + 1, roi)
        original_array[y-1:y+2, x-1:x+2] = roi_plus1

        # Check for 10's at each flash stage if it caused any cascading effect
        check_10 = np.where(original_array == 10)
        # Set found 10's to 0
        original_array = np.where(original_array == 10, 0, original_array)
        # Store loc in tuple
        cascade_flashed = (np.append(cascade_flashed[0],check_10[0]),np.append(cascade_flashed[1],check_10[1]))

    return cascade_flashed, flash_count, original_array

def count_flash(data :pd.DataFrame, steps = 100) -> int:

    # Data prep.
    numbers = [str(no[0]) for no in data.values.tolist()]
    original_array = []
    sync_flash = []
    for idx in range(len(numbers)):
        original_array.append([int(i) for i in numbers[idx]])
    original_array = np.array(original_array)
    original_array = cv2.copyMakeBorder(original_array, 1,1,1,1,cv2.BORDER_CONSTANT, value=[11])

    flash_count = 0
    for step in range(steps):
        original_array = np.array(original_array) + 1
        coord = np.where(original_array == 10)
        while coord[0].size > 0:
            coord, count, original_array = oct_flash(original_array, coord)
            flash_count += count

        if np.all(original_array[1:-1,1:-1] == 0):
            sync_flash.append([step + 1])

    # Remove padding
    original_array = original_array[1:-1,1:-1]
    return flash_count, original_array, sync_flash

In [55]:
data = pd.read_csv('input_data.csv', header=None)
fls, ary, sync = count_flash(data, steps = 1000)
sync[0]

[251]